# Section 2-2 - SVM with Parameter Tuning

Now we simply use Support Vector Machines with parameter tuning using GridSearch SV.

## Pandas - Extracting data

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('../data/train.csv')

## Pandas - Cleaning data

In [2]:
df = df.drop(['Name', 'Ticket', 'Cabin'], axis=1)

age_mean = df['Age'].mean()
df['Age'] = df['Age'].fillna(age_mean)

from scipy.stats import mode

mode_embarked = mode(df['Embarked'])[0][0]
df['Embarked'] = df['Embarked'].fillna(mode_embarked)

df['Gender'] = df['Sex'].map({'female': 0, 'male': 1}).astype(int)
df = pd.concat([df, pd.get_dummies(df['Embarked'], prefix='Embarked')], axis=1)

df = df.drop(['Sex', 'Embarked'], axis=1)

cols = df.columns.tolist()
cols = [cols[1]] + cols[0:1] + cols[2:]

df = df[cols]

train_data = df.values

/usr/local/lib/python2.7/site-packages/numpy/lib/arraysetops.py:200: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))


## Scikit-learn - Training the model

We build the parameter grid for SVC, noting that the default parameters are C = 1.0, and  gamma = 0.0.

In [3]:
from sklearn.svm import SVC
from sklearn.grid_search import GridSearchCV

parameter_grid = {
    'C': [1., 10.],
    'gamma': [0.1, 1.]
}

grid_search = GridSearchCV(SVC(kernel='linear'), parameter_grid, cv=5, verbose=3, n_jobs=4)
grid_search.fit(train_data[0:,2:], train_data[0:,0])

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=4)]: Done  21 out of  20 | elapsed:   46.0s remaining:   -2.2s
[Parallel(n_jobs=4)]: Done  21 out of  20 | elapsed:   48.5s remaining:   -2.3s
[Parallel(n_jobs=4)]: Done  21 out of  20 | elapsed:   52.1s remaining:   -2.5s
[Parallel(n_jobs=4)]: Done  21 out of  20 | elapsed:  1.0min remaining:   -2.9s
[Parallel(n_jobs=4)]: Done  21 out of  20 | elapsed:  1.2min remaining:   -3.3s
[Parallel(n_jobs=4)]: Done  21 out of  20 | elapsed:  1.3min remaining:   -3.6s
[Parallel(n_jobs=4)]: Done  21 out of  20 | elapsed:  1.6min remaining:   -4.5s
[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:  1.6min finished


[CV] C=1.0, gamma=0.1 ................................................
[CV] C=1.0, gamma=0.1 ................................................
[CV] C=1.0, gamma=0.1 ................................................
[CV] C=1.0, gamma=0.1 ................................................
[CV] ....................... C=1.0, gamma=0.1, score=0.804469 -   2.0s[CV] ....................... C=1.0, gamma=0.1, score=0.810056 -   1.5s[CV] ....................... C=1.0, gamma=0.1, score=0.786517 -   2.2s[CV] ....................... C=1.0, gamma=0.1, score=0.752809 -  10.8s



[CV] C=1.0, gamma=1.0 ................................................
[CV] C=1.0, gamma=0.1 ................................................
[CV] C=1.0, gamma=1.0 ................................................
[CV] C=10.0, gamma=0.1 ...............................................
[CV] ....................... C=1.0, gamma=1.0, score=0.804469 -   2.1s[CV] ....................... C=1.0, gamma=0.1, score=0.785311 -  15.6s[CV] ...

GridSearchCV(cv=5, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=4,
       param_grid={'C': [1.0, 10.0], 'gamma': [0.1, 1.0]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=3)

In [4]:
sorted(grid_search.grid_scores_, key=lambda x: x.mean_validation_score)
grid_search.best_score_
grid_search.best_params_

{'C': 1.0, 'gamma': 0.1}

Similarly choose best parameter

In [5]:
model = SVC(
    kernel='linear', 
    C=grid_search.best_params_['C'], 
    gamma=grid_search.best_params_['gamma'])
model = model.fit(train_data[0:,2:], train_data[0:,0])

## Scikit-learn - Making predictions

In [6]:
df_test = pd.read_csv('../data/test.csv')

df_test = df_test.drop(['Name', 'Ticket', 'Cabin'], axis=1)

df_test['Age'] = df_test['Age'].fillna(age_mean)

fare_means = df.pivot_table('Fare', index='Pclass', aggfunc='mean')
df_test['Fare'] = df_test[['Fare', 'Pclass']].apply(lambda x:
                            fare_means[x['Pclass']] if pd.isnull(x['Fare'])
                            else x['Fare'], axis=1)

df_test['Gender'] = df_test['Sex'].map({'female': 0, 'male': 1}).astype(int)
df_test = pd.concat([df_test, pd.get_dummies(df_test['Embarked'], prefix='Embarked')],
                axis=1)

df_test = df_test.drop(['Sex', 'Embarked'], axis=1)

test_data = df_test.values

output = model.predict(test_data[:,1:])

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:9: FutureWarning: scalar indexers for index type Int64Index should be integers and not floating point


## Pandas - Preparing for submission

In [7]:
result = np.c_[test_data[:,0].astype(int), output.astype(int)]

df_result = pd.DataFrame(result[:,0:2], columns=['PassengerId', 'Survived'])
df_result.to_csv('../results/titanic_2-2.csv', index=False)